In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime

pełną dokumentację funkcji zrobi się później

In [2]:
t = 10000 ##żeby wykluczyć mało znane imiona i nazwiska

##imiona męskie i kobiece
im = pd.read_csv("IM.csv")
im = im[im["LICZBA WYSTĄPIEŃ"] > t] 
ik = pd.read_csv("IK.csv")
ik = ik[ik["LICZBA_WYSTĄPIEŃ"] > t]
ik = ik["IMIĘ_PIERWSZE"]
im = im["IMIĘ PIERWSZE"]

##nazwiska męskie i kobiece
nm = pd.read_csv("NM.csv")
nm = nm[nm["Liczba"] > t]
nk = pd.read_csv("NK.csv")
nk = nk[nk["Liczba"] > t]
nk = nk["Nazwisko aktualne"]
nm = nm["Nazwisko aktualne"]

ulice = pd.read_csv("ULIC.csv", sep=";")
ulice_jaworze = ulice.query('WOJ == 24 & POW == 2 & GMI == 6 & RODZ_GMI == 2') ##bo kod terytorialny jaworza to 24.2.6.2
ulice_jaworze = np.array(ulice_jaworze["NAZWA_1"]) ##i bierzemy ulice

##gry
gry = pd.read_csv("GRY.csv")
gry

,ID,NAZWA,RODZAJ,CENA,CZAS_GRY,MIN_GRACZY,MAX_GRACZY,MIN_WIEK,TURNIEJOWE
0,1,Brass: Birmingham,"ekonomiczna, strategiczna",250,150,2,4,14,0
1,2,Star Wars: Rebelia,"strategiczna, science-fiction",390,210,2,4,14,0
2,3,Wojna o Pierścień: Druga edycja,"przygodowa, fantasy",330,150,2,4,13,0
3,4,Zaginiona Wyspa Arnak,"przygodowa, eksploracyjna",150,90,1,4,12,0
4,5,Zamki Burgundii: BIG BOX,strategiczna,140,60,2,4,12,0
...,...,...,...,...,...,...,...,...,...
95,96,Eurobusiness,"ekonomiczna, rywalizacyjna",30,90,2,5,14,0
96,97,Boop,"logiczna, rywalizacyjna",160,20,2,2,10,0
97,98,Mordercze krewetki,"karciane, towarzyskie",80,10,2,6,7,0
98,99,The binding of Isaac: Four souls,"przygodowa, karciana",190,45,2,4,13,0


In [3]:
def ludzie(mi, ki, mn, kn, ul, pk= 0.516, N=7349 ):
    """
    Funkcja do generowania ludności z Jaworza
    pk = 0.516 - procent kobiet w społeczeństwie
    N = 7349 - (w miarę) aktualny stan ludności w tej urokliwej wsi
    
    """
    
    imie = []
    nazwisko = []
    wiek = []
    adres = []
    tel = []
    
    for n in range(N):
        
        p = random.random()
        
        if p < pk: ##losujemy kobietę
            imie.append(random.choice(ki).title()) ##title z AAAA robi Aaaa (a taką formę imion mamy)
            nazwisko.append(random.choice(kn).title())
        
        else: #losujemy mężczyznę
            imie.append(random.choice(mi).title())
            nazwisko.append(random.choice(mn).title())
        
        p = random.random()
        
        if p < 0.575: ##dane statystyczne - 57.5% osób jest w wieku produkcyjnym
            wiek.append(random.randint(18,60))
        elif p < 0.77:
            wiek.append(random.randint(8, 18)) ##19.4% w przedprodukcyjnym
        else:
            wiek.append(random.randint(60,99)) ##reszta starsza
            
            
        p = random.random()   
        if p< 0.90:
            adres.append(random.choice(ul) + ' ' + str(random.randint(1,100)))
            
            p = random.random()
            if p< 0.90:
                tel.append(random.randint(500000000, 999999999))
            else:
                tel.append("NULL")
                
        else:
            adres.append("NULL")
            tel.append(random.randint(500000000, 999999999))
        
        
            
    
    return pd.DataFrame({"id_mieszkańca": range(1,N+1), "imię": imie, "nazwisko": nazwisko, 
                         "wiek": wiek, "adres": adres, "telefon": tel})

In [4]:
def transform_wiek(wiek): 
    """
    Funkcja zwracająca częstość z jaką dana osoba na podstawie wieku mogłaby nas odwiedzać. 
    Im większa wartość tym większe prawdopodobieństwo wizyty.
    
    """
    
    if wiek < 10:
        return 1
    elif wiek < 12:
        return 2
    elif wiek < 16:
        return 3
    elif wiek < 20:
        return 5
    elif wiek < 30:
        return 6
    elif wiek < 38:
        return 5
    elif wiek < 45:
        return 4
    elif wiek < 55:
        return 3
    elif wiek < 70:
        return 2
    else:
        return 1


In [5]:
def inven(n = 100):
    """
    Funkcja tworząca prosty magazyn (wynajem/sklep). Zwraca ile gier o danym tytule (id_tytułu) jest u nas dostępna na półce.
    n = 100 - bo tyle różnych gier (i takich id) mamy w asortymencie
    """
    
    ilosc = np.random.randint(20, 30, n) ## od 20 do 30 każdego z tytułów
    ids = range(1, (n+1))
    ilosci = [ids[x] for x in range(len(ids)) for i in range(ilosc[x])]
    
    return pd.DataFrame({"id_inv": range(1, (len(ilosci)+1)),"id_tytuł": ilosci })

In [6]:
ludzie(im, ik, nm, nk, ulice_jaworze)

,id_mieszkańca,imię,nazwisko,wiek,adres,telefon
0,1,Bernadeta,Sobczak,25,Pałacowa 60,695993350
1,2,Ludwik,Jankowski,51,Kaczeńcowa 96,594965517
2,3,Rozalia,Piątek,42,Kryształowa 76,671701133
3,4,Marlena,Włodarczyk,21,Dziewanny 33,NULL
4,5,Lidia,Wiśniewska,9,Kalinowa 56,524158663
...,...,...,...,...,...,...
7344,7345,Elżbieta,Kowalczyk,60,Różana 9,548064795
7345,7346,Kinga,Rutkowska,91,Średnia 53,NULL
7346,7347,Kaja,Borowska,64,Dębowa 34,NULL
7347,7348,Janina,Brzozowska,31,Słonecznikowa 45,760150675


In [7]:
def odwiedziny(start = datetime.datetime(2022, 6, 6, 9, 0, 0), lamb = 4):
    """
    Funkcja symulująca odwiedziny w naszym sklepie - sprzedaże, wynajmy, a także zakup w outlecie. 
    Zwraca godziny wizyt, mieszkańców, co kupili/ wynajęli.
    start - dzień startowy naszego przedsiębiorstwa, niech to będzie 6 czerwca 2022, 9:00
    lamb - lambda - parametr potrzebny do generowania procesu Poissona - zakładamy 4 klientów na godzinę w naszym sklepie
   
    """
    
    mieszkancy = ludzie(im, ik, nm, nk, ulice_jaworze) #baza mieszkańców, którzy mogą nas odwiedzić
    T = 239/30 #7h 58 min bo tyle nasz sklep jest otwarty (z małym buforem, żeby nikt nie kupił czegoś po 1 s od otwarcia)
    
    now = datetime.datetime.now()
    days = (now - start).days ##żeby te daty były do dzisiaj
    
    
    inv_ren = inven() ##startowy magazyn wynajmu
    inv_shop = inven() ##startowy magazyn sklepu
    pp = [transform_wiek(mieszkancy["wiek"][x]) for x in range(len(mieszkancy))] ##wagi dla mieszkancow do losowania
    
    df = pd.DataFrame({}) ##podstawowa tabela, do której dodawane będą wizyty
    rental =  pd.DataFrame({"id_inv": [], "return_date": [], "zniszczona": []}) ##tabela pomocnicza dla inv rentalowego
    kupno =  pd.DataFrame({"id_inv": [], "kupno": [], "wizyta":[]}) ##dla kupna
    outlet = pd.DataFrame({"id_inv": [], "id_gry": [], "return_date": []}) ##dla outletu 
    arch_outlet =  pd.DataFrame({"id_inv": [], "date": []})
    
    swieta = [datetime.datetime(2023, 1, 6, 9, 0, 0), datetime.datetime(2023, 4, 9, 9, 0, 0), 
              datetime.datetime(2023, 4, 10, 9, 0, 0), datetime.datetime(2023, 3, 1, 9, 0, 0), 
              datetime.datetime(2023, 3, 3, 9, 0, 0), datetime.datetime(2023, 6, 8, 9, 0, 0), 
              datetime.datetime(2022, 6, 16, 9, 0, 0), datetime.datetime(2022, 8, 15, 9, 0, 0),
              datetime.datetime(2022, 11, 1, 9, 0, 0), datetime.datetime(2022, 11, 11, 9, 0, 0),
              datetime.datetime(2022, 12, 25, 9, 0, 0), datetime.datetime(2022, 12, 26, 9, 0, 0),]
    dd = [(s - start).days + 1 for s in swieta] ##dni jakie minęły od startu do święta (żeby móc je ominąć łatwo)
    
    exp = np.ceil(np.random.exponential(1, 150)) ##z tego będą losowane ilości gier kupionych/wynajętych
    
    for d in range(1, days + 1):
        
        if d%6 == 0 or d%7 == 0 or (d in dd): ## wyłączenie weekendów oraz świąt
            pass
        
        else:
            ##odwiedziny zgodne z procesem poissona
            N = np.random.poisson(T*lamb) ##ilość osób na cały dzień
            ts = sorted(T*np.random.uniform(0,1, N)) ##czasy wizyt (takie surowe)
            t_wizyty = [(start + datetime.timedelta(minutes=1) + datetime.timedelta(
                minutes=round(t*60,1))) for t in ts] #.strftime("%m/%d/%Y %H:%M:%S") for t in ts]
            ##formatka wizyt
            
            for t in t_wizyty:
                ##losowanie klientów z populacji mieszkańców
                klient = random.choices(mieszkancy["id_mieszkańca"], weights=pp , k=1)
            
                #losowanie sprzedawcy (zakładamy, że sprzedawca 1 i 2 obsługują kasy przez te 8 h)
                sprzedawca = random.choices([1,2], k=1)
            
                ##ktoś może chce kupić kilka gier zamiast jednej
                ilosc = random.choices(exp, k = 1)
            
                ##ktoś przychodzi z zamiarem kupna/wynajmu bądź skorzystania z outletu (jeśli może)
                ##ilość dostępnych gier w outlecie musi wynosić min tyle ile ktoś chce tych gier zakupić
                
                if len(outlet[outlet["return_date"] < start]) >= ilosc[0]: 
                    rodzaj = random.choices(["wynajem", "kupno", "outlet"], weights = [10, 10, 2], k = 1)
                    ##ustalamy prawdopodobieństwa skorzystania z outletu na 2:10:10 względem zwykłego kupna i wynajmu
                else:
                    rodzaj = random.choices(["wynajem", "kupno"], k = 1)
                    #jeśli outlet jest za słabo wyposażony zostaje tylko opcja wynajmu / kupna z wagami 1:1
                
                if rodzaj == ["wynajem"]:
                    
                    ##losuje zwrot za 1 lub 2 dni w godzinach otwarcia, bo zakładamy, że tyle mają czasu na zwrot
                    return_date = start + datetime.timedelta(days=random.randint(1,2)) + datetime.timedelta(
                        hours=round(random.uniform(1,T), 2))
                    
                    ##losuje id_gry, które ktoś wypożycza
                    ##szukamy tych id które są dostępne, czyli NIE ma ich w df rental z returndate > dziś
                    
                    not_in_use = np.array(rental[rental["return_date"] > start]["id_inv"]) ##niedostępne
                    ids = np.array(inv_ren["id_inv"]) ##wszystkie
                    in_use = np.setdiff1d(ids, not_in_use) ##różnica między wszystkimi a tymi niedostępnymi
                    gra = random.sample(sorted(in_use), k = int(ilosc[0])) ##z tych dostepnych losowanie gry
                    
                    ##ewentualne zniszczenia zwróconej gry - prawdopodobieństwo ustalamy na 1/100
                    fault = random.choices([0,1], weights = [99,1], k = int(ilosc[0])) 
                    
                    ##wrzucenie informacji do tymczasowej tabeli rental
                    rent_gry =  pd.DataFrame({"id_inv": gra, "return_date": np.repeat(return_date, int(ilosc[0])), 
                                              "zniszczona": fault})
                    rental = pd.concat([rental, rent_gry])
                    
                    ## gra może zostać zniszczona, a więc może trafić do outletu
                    if sum(fault) > 0: ##czy były jakieś zniszczenia - jeśli tak to update outletu
                        
                        idd = rent_gry[rent_gry["zniszczona"] == 1]["id_inv"] ##te zniszczone id_inv
                        
                        for i in idd: ##dla każdej zniszczonej gry
                            ##znajdujemy id_tytułu (bo działaliśmy na inv)
                            id_tyt = inv_ren.loc[inv_ren['id_inv'] == i]["id_tytuł"] 
                
                            f = pd.DataFrame({"id_inv": i, "id_gry": id_tyt, "return_date": return_date})
                            outlet = pd.concat([outlet, f]) ## i dodajemy gre do outletu
                    
                elif rodzaj == ["kupno"]: 
                    ##to samo co z rentalem, tylko patrzymy na id gry, które ktoś kupił DZISIAJ 
                    ##bo zakładamy, że kupiona dzisiaj gra, dostępna będzie ponownie jutro
                    
                    return_date = None
                    not_in_use2 = np.array(kupno[kupno["kupno"] == start]["id_inv"])
                    ids2 = np.array(inv_shop["id_inv"])
                    in_use2 = np.setdiff1d(ids2, not_in_use2)
                    gra = random.sample(sorted(in_use2), k = int(ilosc[0]))
                
                    kup_gry =  pd.DataFrame({"id_inv": gra, "kupno": np.repeat(start, int(ilosc[0])),
                                             "wizyta": np.repeat(t, int(ilosc[0]))})
                    kupno = pd.concat([kupno, kup_gry])
                    
                else: ##outlet
                    
                    return_date = None
                    in_use = outlet[outlet["return_date"] < start] ##patrzymy na te już zwrócone i dostepne gry
                    gra = random.sample(sorted(in_use["id_inv"]), k = int(ilosc[0])) ##losowanie z dostepnego outletu
                    ##id_inv odnoszą się na razie do magazynu wynajmu, ale później odnosić się będą do swojego id
                    ##to w formatce tabel
                    
                    for g in gra:
                        outlet = outlet.drop(outlet.index[outlet["id_inv"] == g])
                        ##usuwanie z outletu kupionych pozycji
                        arch = pd.DataFrame({"id_inv": g, "date": t}, index=[0])
                        arch_outlet = pd.concat([arch_outlet, arch])
                        
                ##dołączenie danego klienta do df z poprzednich dni
                day = pd.DataFrame({"id_mieszkańca": klient, "wizyta": t,
                                "sprzedawca": sprzedawca, "ilosc": int(ilosc[0]), "id_gry": str(gra)[1:-1], "rodzaj": rodzaj,
                                   "return_date": return_date})
                df = pd.concat([df, day])
            
        ##przejście z datą na kolejny dzień
        start = start + datetime.timedelta(days=1)
        
    return df, rental, kupno, outlet, inv_ren, inv_shop, mieszkancy, arch_outlet

In [8]:
def format_tabel(wizyty, gry = gry):
    """
    Funkcja służąca do formatowania otrzymanych wyników z funkcji odwiedziny() w taki sposób, żeby zwracała
    gotowe tabele, które trafią do bazy danych.
    wizyty - wynik z funkcji odwiedziny()
    gry - tabela z grami naszego wykonania
    
    """
    tabela_g = wizyty[0] ##tabela główna
    rental_arch = wizyty[1] ##archiwum wynajmu
    shop_arch = wizyty[2] ##archiwum sprzedaży
    outlet_akt = wizyty[3] ##aktualny outlet
    rent_inv = wizyty[4] ##inventory rental
    shop_inv = wizyty[5] ##inventory shop
    mieszkancy = wizyty[6] ##cała społeczność
    outlet_arch = wizyty[7] ## archiwum outletu
    
    tabela_g = tabela_g.rename(columns = {"sprzedawca":"id_pracownika"})
    ## ------------- TABELA KLIENCI -------------
    
    #szukamy unikatowych osób, które pojawiły się u nas w sklepie i ich ostatnią wizytę
    klient = tabela_g.drop_duplicates(subset=['id_mieszkańca'], keep="last")[["id_mieszkańca", "wizyta"]]
    klienci = pd.merge(klient, mieszkancy, how = "inner", on = ["id_mieszkańca"]) ##dopasowanie względem id_mieszkanca
    klienci["id_klienta"] = range(1, len(klienci) + 1) ##stworzenie id klienta
    
    klienci_mieszkancy = klienci[["id_klienta", "id_mieszkańca"]] ##tabela pomocnicza z id klienta dopasowanymi do mieszkańca
    
    klienci = klienci[["id_klienta", "wizyta", "imię", "nazwisko", "wiek", "adres", "telefon"]] #końcowa tabela klientów
    
    ## ---------------- TABELA SPICHLERZ_WYNAJEM ---------------
    ceny = gry[["ID","CENA"]]
    ceny = ceny.rename(columns = {"ID": "id_tytuł", "CENA": "cena"}) ##tymczasowa tabela z cenami
    ceny["cena_wynajem"] = np.round(0.15*ceny["cena"], 2 ) ###bo ustalamy, że cena wynajmu gry kosztuje 15% jej ceny rynkowej
    
    one = pd.merge(rent_inv, ceny, on="id_tytuł") ##krok pierwszy - połączenie wynajmu z cenami najmu
    ids = np.array(rental_arch["id_inv"])
    rental_arch["id_inv"] = [int(i) for i in ids]  ##po random sample zostały float jako typ id, więc wracam na int
    two = rental_arch.drop_duplicates(subset=['id_inv'], keep='last') ##biorę pod uwagę tylko ostatnie wypożyczenie danej gry
    
    spichlerz_wynajem = pd.merge(one,
                             two[["id_inv","return_date"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_wynajem", "id_tytuł": "id_gry", "return_date": "ostatni_update"})
    ##i ostateczna tabela:
    spichlerz_wynajem = spichlerz_wynajem[["id_spichlerz_wynajem", "id_gry", "cena_wynajem", "ostatni_update"]]
    
    ## ------------------- TABELA SPICHLERZ_SKLEP ---------------
    
    ceny["cena_kupno"] = np.round(1.1*ceny["cena"], 2 ) ###bo ustalamy marżę 10% na zakupie
    raz = pd.merge(shop_inv, ceny, on="id_tytuł")[["id_tytuł", "cena_kupno", "id_inv"]] ##znowu pierwsze merge z cenami
    
    ids2 = np.array(shop_arch["id_inv"])
    shop_arch["id_inv"] = [int(i) for i in ids2] ##znowu zmiana na int
    shop_arch["kupno"] = [d + datetime.timedelta(days=1) for d in shop_arch["kupno"]] ## bo gra jest dostepna 1 dzień po zakupie
    dwa = shop_arch.drop_duplicates(subset=['id_inv'], keep='last') ##znowu patrzymy na ostatnie zakupy
    spichlerz_sklep = pd.merge(raz,
                             dwa[["id_inv","kupno"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_sklep", "id_tytuł": "id_gry", "kupno": "ostatni_update"})
    spichlerz_sklep = spichlerz_sklep[["id_spichlerz_sklep", "id_gry", "cena_kupno", "ostatni_update"]] ##ostatnia formatka
    
    ## ---------------- TABELA WYNAJEM ---------
    sub = pd.merge(rental_arch, tabela_g, on = ["return_date"])[["id_inv", "wizyta", "return_date", "zniszczona", "id_mieszkańca", "id_pracownika"]]
    ##łączymy archiwum z główną tabelą - mamy teraz więcej informacji i dodajemy id klienta
    wynajemt = pd.merge(sub, klienci_mieszkancy, how = "left", on = ["id_mieszkańca"]).rename(columns={"id_inv":"id_spichlerz_wynajem"})
    ## i merge tabeli tymczasowej ze spichlerzem, żeby uzyskać cenę
    wynajem = pd.merge(wynajemt, spichlerz_wynajem[["id_spichlerz_wynajem","cena_wynajem"]], on=["id_spichlerz_wynajem"], how="left")
    ##i ostateczna forma tabeli:
    wynajem =  wynajem.sort_values(by = "wizyta", inplace=False, ascending=True) ##sort po datach
    wynajem = wynajem.rename(columns = {"wizyta": "data_wynajmu", "return_date": "data_zwrotu"}) ##szybki rename
    wynajem["id_transakcji_wynajem"] = range(1, len(wynajem) + 1) ##dodanie id_transakcji
    wynajem = wynajem[["id_transakcji_wynajem", "id_spichlerz_wynajem", "data_wynajmu", "data_zwrotu", "id_pracownika", "id_klienta", "zniszczona"]]
    
    ## ----------- TABELA SKLEP --------------------
    
    shop_arch['wizyta'] = pd.to_datetime(shop_arch['wizyta']) ##merge z tabelą główną
    sklept = pd.merge(shop_arch, tabela_g, on="wizyta", how="left")[["id_inv", "wizyta", "id_mieszkańca", "id_pracownika"]]
    sklep = pd.merge(sklept, klienci_mieszkancy, how="left", on = ["id_mieszkańca"]).rename(columns = {"id_inv": "id_spichlerz_sklep"})
    sklep["id_transakcji_sklep"] = range(1, len(sklep) + 1)
    sklep = sklep[["id_transakcji_sklep", "id_spichlerz_sklep", "wizyta", "id_klienta", "id_pracownika"]]
    
    
    ## -------------- TABELA OUTLET ----------------
    
    outlet_arch['date'] = pd.to_datetime(outlet_arch['date'])
    ids = outlet_arch["id_inv"]
    outlet_arch["id_inv"] = [int(i) for i in ids]
    outlet_arch = outlet_arch.rename(columns = {"id_inv":"id_spichlerz_wynajem", "date":"wizyta"})
    out = pd.merge(outlet_arch, tabela_g, on="wizyta", how="left")[["id_spichlerz_wynajem", "wizyta", "id_mieszkańca", "id_pracownika"]]
    out2 = pd.merge(out, spichlerz_wynajem[["id_spichlerz_wynajem", "id_gry", "cena_wynajem"]], on="id_spichlerz_wynajem", how="left")
    out2["cena_wynajem"] = [3* c for c in out2["cena_wynajem"]] 
    ##bo ustalamy 45% czyli 3* cenę najmu jako cenę kupna outletowej gry
    outlett = pd.merge(out2, klienci_mieszkancy, how="left", on = ["id_mieszkańca"])
    outlett["id_transakcji_outlet"] = range(1, len(outlett) + 1)
    outlet = outlett[["id_transakcji_outlet", "id_spichlerz_wynajem", "id_gry", "wizyta", "id_klienta", "id_pracownika"]]
    
    return klienci, spichlerz_wynajem, spichlerz_sklep, wynajem, sklep, outlet

In [9]:
tt = odwiedziny()

In [10]:
tab = format_tabel(tt)
klienci = tab[0]
spichlerz_wynajem = tab[1]
spichlerz_sklep = tab[2]
wynajem = tab[3]
sklep = tab[4]
outlet = tab[5]

In [19]:
spichlerz_wynajem

,id_spichlerz_wynajem,id_gry,cena_wynajem,ostatni_update
0,1,1,37.5,2023-02-18 13:04:12
1,2,1,37.5,2023-02-23 13:33:00
2,3,1,37.5,2023-05-04 13:14:24
3,4,1,37.5,2023-01-03 14:28:48
4,5,1,37.5,2023-04-28 11:46:12
...,...,...,...,...
2454,2455,100,9.0,2022-12-31 14:34:12
2455,2456,100,9.0,2022-11-25 15:04:12
2456,2457,100,9.0,2023-05-10 13:01:12
2457,2458,100,9.0,2023-03-23 16:08:24


In [29]:
id_wynajem = np.array(spichlerz_wynajem[spichlerz_wynajem['id_gry']==62]['id_spichlerz_wynajem'])
id_wynajem

array([1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510,
       1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520],
      dtype=int64)

In [38]:
przedzial = wynajem[np.isin(wynajem['id_spichlerz_wynajem'],id_wynajem)][['data_wynajmu','data_zwrotu']]
przedzial['data_wynajmu'].iloc[0]

Timestamp('2022-06-06 14:23:36')

In [51]:
start = datetime.datetime(2023, 3, 20, 18, 0, 0)

In [52]:
i = 0
for _ in range(len(przedzial)):
    if przedzial.iloc[_]['data_wynajmu'] < start < przedzial.iloc[_]['data_zwrotu']:
        i=+1

In [53]:
print(i)

0


In [56]:
def turniej(gracze,stoly,gry = gry,inv = spichlerz_wynajem, rental = wynajem):
    start = datetime.datetime(2022, 6, 9, 18, 0, 0)
    now = datetime.datetime.now()
    days = (now - start).days ##żeby te daty były do dzisiaj

    two_weeks = datetime.timedelta(weeks = 2)
    future = now + two_weeks*6
    # generowanie dat turniejów
    all_date_tournament = np.array([start])
    while all_date_tournament[-1]<future:
        next_tournament = all_date_tournament[-1]+two_weeks
        all_date_tournament = np.append(all_date_tournament,next_tournament)

    # sprawdzamy czy daty kolidują z dniami wolnymi
    swieta = [
              datetime.datetime(2023, 1, 6, 18, 0, 0), datetime.datetime(2023, 4, 18, 9, 0, 0), 
              datetime.datetime(2023, 4, 10, 18, 0, 0), datetime.datetime(2023, 3, 1, 18, 0, 0), 
              datetime.datetime(2023, 3, 3, 18, 0, 0), datetime.datetime(2023, 6, 8, 18, 0, 0), 
              datetime.datetime(2022, 6, 16, 18, 0, 0), datetime.datetime(2022, 8, 15, 18, 0, 0),
              datetime.datetime(2022, 11, 1, 18, 0, 0), datetime.datetime(2022, 11, 11, 18, 0, 0),
              datetime.datetime(2022, 12, 25, 18, 0, 0), datetime.datetime(2022, 12, 26, 18, 0, 0),]

    all_date_tournament = np.delete(all_date_tournament,np.isin(all_date_tournament,swieta))

    date_tournament = all_date_tournament[all_date_tournament<now]

    
    # spis_turnieju
    gra_tournament = gry[gry['TURNIEJOWE'] == 1]
    id_spis = [i for i in range(1,len(gra_tournament)+1)]
    # zakładam, iz czsas dodatkowy to 0.3
    # sprawdzić czy na stanie jest wytarczająca ilość gier
    amount = np.floor(240/(np.array(gra_tournament[['CZAS_GRY']])+np.array(gra_tournament[['CZAS_GRY']])*0.3))*stoly
    amount = amount.transpose()
    # graczy
    min_graczy = np.array(gra_tournament['MAX_GRACZY'])/2*amount.astype(int)
    max_graczy = np.array(gra_tournament['MAX_GRACZY'])*amount.astype(int)

    # średnia z punktów dla poszczególnych gier
    mean_point = np.array([110,50,350])
    # tworzenie dataframe
    spis_turniej = pd.DataFrame({'id_spis':id_spis,'id_gry':gra_tournament['ID'],'średnia_punktów' : mean_point,
                            'ilosc_gier' : amount[0], 'min_graczy': min_graczy[0], 'max_graczy':max_graczy[0]})
    # turniej
    id_turniej = [i for i in range(1,len(all_date_tournament)+1)]
    kind_tournament = random.choices(np.array(id_spis),k=len(all_date_tournament)) # losowanie typu turnieju
    date_date = [dt.date() for dt in all_date_tournament] # tylko daty
    rozgrywka = pd.DataFrame({'id_turnieju':id_turniej, 'id_rodzaj':kind_tournament,'data':date_date})
# wyniki

    id_turnieji = np.array([])
    id_klientow = np.array([])
    wyniki = np.array([])
    czas_rozrywki = np.array([])
    for _ in range(len(date_tournament)):

        kt = kind_tournament[_]
        idgry = np.array(spis_turniej[spis_turniej['id_spis']==kt][['id_gry']].iloc[0])[0]
        # sprawdzam czy jest wystarczająca ilosc gier
        id_wynajem = np.array(inv[inv['id_gry']==idgry]['id_spichlerz_wynajem'])
        przedzial = rental[np.isin(rental['id_spichlerz_wynajem'],id_wynajem)][['data_wynajmu','data_zwrotu']]
        i = 0
        for j in range(len(przedzial)):
            if przedzial.iloc[j]['data_wynajmu'] < date_tournament[_] < przedzial.iloc[j]['data_zwrotu']:
                i=+1
        
        if id_wynajem.shape[0] - i < np.array(spis_turniej[spis_turniej['id_spis']==kt][['ilosc_gier']].iloc[0])[0]:
            raise ValueError('No to chop na gałąź')

        ig = np.array(spis_turniej[spis_turniej['id_spis']==kt][['min_graczy','max_graczy']].iloc[0]) # wczytywanie min max graczy
        
        #il = np.array(gry[gry['ID']==ig][['MIN_GRACZY','MAX_GRACZY']])
        #il_max = il[0][1]

        # ilość graczy na turnieju
        il = random.randint(ig[0],ig[1])
        #n = sum([random.randint(il[0][0],il[0][1]) for i in range(int(turniej[turniej['id_turnieju']==kt][['ilosc_gier']].iloc[0]))])
        #n = int(turniej[turniej['id_turnieju']==kt]['ilosc_gier'].iloc[0]/2)

        # znajdowanie max graczy w grze(planszy)
        max_g = np.array(gra_tournament[gra_tournament['ID']==idgry]['MAX_GRACZY'])[0]
        
        if il%(max_g) == 0 :
            for n_ in range(int(il/(max_g))):
                
                id_rozgrywki_s = np.round(np.array([id_turniej[_]]*(max_g)),0) # powtarzamy id_turnieju 
                id_turnieji = np.append(id_turnieji,id_rozgrywki_s)

                id_klienta = random.sample(list(gracze['id_klienta']),k = max_g) # losujemy graczy bez zwracania
                id_klientow = np.append(id_klientow,id_klienta)

                wynik = [spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]]*max_g + np.random.normal(
                        scale=spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]*0.15,size=max_g)
                wyniki = np.append(wyniki,wynik) # losujemy wynik
                
                czas = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*max_g,0)
                #czas_2 = np.round([gra_tournament[gra_tournament['ID']==ig]['CZAS_GRY'].iloc[0] + np.random.normal(scale=10)]*il_max,0)

                #czas = np.append(czas_1,czas_2)
                czas_rozrywki = np.append(czas_rozrywki,czas)
        elif il%max_g == 1 or il%max_g == 2:
            vec = range(np.ceil(il/max_g).astype(int))
            for _n in vec:
                if _n in vec[:-2]:
                    id_rozgrywki_s = np.round(np.array([id_turniej[_]]*(max_g)),0) # powtarzamy id_turnieju 
                    id_turnieji = np.append(id_turnieji,id_rozgrywki_s)

                    id_klienta = random.sample(list(gracze['id_klienta']),k = max_g) # losujemy graczy bez zwracania
                    id_klientow = np.append(id_klientow,id_klienta)

                    wynik = [spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]]*max_g + np.random.normal(
                            scale=spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]*0.15,size=max_g) # losujemy wynik
                    wyniki = np.append(wyniki,wynik) 

                    # czas rozgrywki jest wyznaczamy dla graczy przy jednej planszy
                    czas = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*max_g,0) 
                    czas_rozrywki = np.append(czas_rozrywki,czas)

                elif _n == vec[-1]:
                    pass
                else:
                    gracze_1 = np.floor((il-max_g*(_n))/2).astype(int) # ilosc graczy przy 1 planszy
                    gracze_2 = (il-max_g*(_n)-gracze_1).astype(int) # ilosc graczy przy 2 planszy

                    id_rozgrywki_s = np.round(np.array([id_turniej[_]]*(il-max_g*(_n))),0) # powtarzamy id_turnieju 
                    id_turnieji = np.append(id_turnieji,id_rozgrywki_s)

                    id_klienta = random.sample(list(gracze['id_klienta']),k = (il-max_g*(_n))) # losujemy graczy bez zwracania
                    id_klientow = np.append(id_klientow,id_klienta)

                    wynik = [spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]]*(il-max_g*(_n)) + np.random.normal(
                            scale=spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]*0.15,size=(il-max_g*(_n)))
                    wyniki = np.append(wyniki,wynik) # losujemy wynik

                    czas_1 = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*gracze_1,0)
                    czas_2 = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*gracze_2,0)

                    czas = np.append(czas_1,czas_2)
                    czas_rozrywki = np.append(czas_rozrywki,czas)
        else:
            vec = range(np.ceil(il/max_g).astype(int))
            for _n in vec:
                if _n in vec[:-1]:
                    id_rozgrywki_s = np.round(np.array([id_turniej[_]]*(max_g)),0) # powtarzamy id_turnieju 
                    id_turnieji = np.append(id_turnieji,id_rozgrywki_s)
                    
                    id_klienta = random.sample(list(gracze['id_klienta']),k = max_g) # losujemy graczy bez zwracania
                    id_klientow = np.append(id_klientow,id_klienta)

                    wynik = [spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]]*max_g + np.random.normal(
                            scale=spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]*0.15,size=max_g)
                    wyniki = np.append(wyniki,wynik) # losujemy wynik

                    czas = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*max_g,0)
                    czas_rozrywki = np.append(czas_rozrywki,czas)
                else:

                    id_rozgrywki_s = np.round(np.array([id_turniej[_]]*(il-max_g*(_n))),0) # powtarzamy id_turnieju 
                    id_turnieji = np.append(id_turnieji,id_rozgrywki_s)
                    
                    id_klienta = random.sample(list(gracze['id_klienta']),k = (il-max_g*(_n))) # losujemy graczy bez zwracania
                    id_klientow = np.append(id_klientow,id_klienta)

                    wynik = [spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]]*(il-max_g*(_n)) + np.random.normal(
                            scale=spis_turniej[spis_turniej['id_spis']==kt]['średnia_punktów'].iloc[0]*0.15,size=(il-max_g*(_n)))
                    wyniki = np.append(wyniki,wynik) # losujemy wynik

                    czas = np.round([gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0] + np.random.normal(
                           scale= gra_tournament[gra_tournament['ID']==idgry]['CZAS_GRY'].iloc[0])*0.1]*(il-max_g*_n),0)
                    czas_rozrywki = np.append(czas_rozrywki,czas)


    print(len(id_turnieji),len(id_klientow),len(wyniki))
    df_wynik = pd.DataFrame({'id_turniej':id_turnieji.astype(int),'id_klienta':id_klientow.astype(int),'wynik':np.ceil(wyniki).astype(int),'czas_rozgrywki':czas_rozrywki.astype(int)})    
    return spis_turniej, rozgrywka, df_wynik  

In [58]:
n, m,p = turniej(klienci,2)

974 974 974


In [16]:
m

,id_turnieju,id_rodzaj,data
0,1,1,2022-06-09
1,2,3,2022-06-23
2,3,3,2022-07-07
3,4,3,2022-07-21
4,5,1,2022-08-04
5,6,3,2022-08-18
6,7,3,2022-09-01
7,8,2,2022-09-15
8,9,3,2022-09-29
9,10,3,2022-10-13


In [ ]:
gra_tournament = gry[gry['TURNIEJOWE'] == 1]
gra_tournament

,ID,NAZWA,RODZAJ,CENA,CZAS_GRY,MIN_GRACZY,MAX_GRACZY,MIN_WIEK,TURNIEJOWE
61,62,Wsiąść do Pociągu: Europa,"strategiczna, familijna",140,30,2,5,8,1
64,65,Carcassonne,strategiczna,70,30,2,5,7,1
90,91,Scrabble,słowna,110,90,2,4,10,1


In [ ]:
czas = np.round([gra_tournament[gra_tournament['ID']==62]['CZAS_GRY'].iloc[0] + np.random.normal(scale=10)]*(4),0)
czas

array([19., 19., 19., 19.])

In [ ]:
outlet ##nie wiem czy zostawiać tu id inventorowe czy id gry

,id_transakcji_outlet,id_spichlerz_wynajem,id_gry,wizyta,id_klienta,id_pracownika
0,1,113,5,2022-06-09 09:25:12,3500,1
1,2,2284,92,2022-06-21 13:00:18,87,2
2,3,236,10,2022-07-12 13:36:36,239,2
3,4,2177,88,2022-07-16 09:50:36,3796,1
4,5,1673,68,2022-07-18 12:12:12,284,2
...,...,...,...,...,...,...
56,57,621,25,2023-05-20 10:00:06,3868,1
57,58,2182,88,2023-05-22 09:15:48,3884,1
58,59,2470,100,2023-05-29 11:22:54,4022,2
59,60,342,14,2023-06-13 10:42:12,4260,2


In [ ]:
tt[0][tt[0]["rodzaj"] == "outlet"]

,id_mieszkańca,wizyta,sprzedawca,ilosc,id_gry,rodzaj,return_date
0,2665,2022-06-09 09:25:12,1,1,113.0,outlet,None
0,5451,2022-06-21 13:00:18,2,1,2284.0,outlet,None
0,777,2022-07-12 13:36:36,2,1,236.0,outlet,None
0,6562,2022-07-16 09:50:36,1,1,2177.0,outlet,None
0,4433,2022-07-18 12:12:12,2,1,1673.0,outlet,None
0,3659,2022-07-19 09:33:30,2,1,1007.0,outlet,None
0,5069,2022-07-20 09:08:00,1,1,80.0,outlet,None
0,1328,2022-07-26 12:21:42,1,1,2126.0,outlet,None
0,168,2022-07-26 12:21:54,2,1,1553.0,outlet,None
0,486,2022-08-09 09:27:12,1,1,268.0,outlet,None


In [ ]:
klienci

,id_klienta,wizyta,imię,nazwisko,wiek,adres,telefon
0,1,2022-06-06 09:56:48,Vadym,Majewski,17,NULL,838865200
1,2,2022-06-06 10:20:48,Bernadeta,Kania,10,Grabowa 46,876487052
2,3,2022-06-06 10:29:54,Robert,Kamiński,38,Kryształowa 17,829590296
3,4,2022-06-06 11:44:30,Przemysław,Bąk,81,Rodzinna 33,863961726
4,5,2022-06-06 12:56:18,Tomasz,Tomczak,38,Bratków 39,NULL
...,...,...,...,...,...,...,...
4501,4502,2023-06-22 15:48:06,Mikołaj,Markiewicz,21,Nadbrzeżna 26,801289703
4502,4503,2023-06-22 16:06:36,Ignacy,Mikołajczyk,35,Modrzewiowa 66,827039170
4503,4504,2023-06-22 16:14:12,Bruno,Nowak,37,NULL,509292454
4504,4505,2023-06-22 16:23:54,Lech,Pawlik,19,Nagietkowa 29,996284617


In [ ]:
sklep

,id_transakcji_sklep,id_spichlerz_sklep,wizyta,id_klienta,id_pracownika
0,1,1351,2022-06-06 09:35:06,3375,2
1,2,437,2022-06-06 09:56:06,810,2
2,3,274,2022-06-06 09:56:06,810,2
3,4,2026,2022-06-06 09:56:06,810,2
4,5,1454,2022-06-06 10:09:12,4134,1
...,...,...,...,...,...
6693,6694,757,2023-06-22 14:55:12,4500,2
6694,6695,1731,2023-06-22 15:48:06,4502,1
6695,6696,20,2023-06-22 15:48:06,4502,1
6696,6697,201,2023-06-22 16:23:54,4505,1


##  tabele sprzedawców

In [ ]:
def payment(rola):
    if rola == "sprzedawca":
        return random.randint(3700, 4300)
    elif rola == "logistyk":
        return random.randint(5000, 6000)
    else:
        return random.randint(6000, 7000)

In [ ]:
def sales(ul = ulice_jaworze):
    role = ["sprzedawca", "sprzedawca", "manager", "logistyk"]
    id_pracownika = [1,2,3,4]
    pay_s = payment("sprzedawca")
    pay_m = payment("manager")
    pay_l = payment("logistyk")
    pensje = [pay_s, pay_s, pay_m, pay_l]
    
    imiona = ["Misio", "Nati", "Alutka", "Fifol"]
    nazwiska = ["Bresiński", "Lach", "Myśliwiec", "Oszczepaliński"]
    wiek = [22, 22, 21, 21]
    r = random.sample(id_pracownika, k=4)
    
    adresy = [random.choice(ul) + ' ' + str(random.randint(1,100)) for _ in range(4)]
    telefony = [random.randint(500000000, 999999999) for _ in range(4)]
    
    one = pd.DataFrame({"id_pracownika" : r, "imię_pracownika" : imiona, 
                        "nazwisko_pracownika" : nazwiska, "wiek_pracownika": wiek,
                       "adres_pracownika" : adresy, "telefon_pracownika" : telefony})
    two = pd.DataFrame({"id_pracownika": id_pracownika, "rola": role, "pensja":pensje})
    
    pracownicy = pd.merge(one, two, on="id_pracownika")
    pracownicy = pracownicy.sort_values(by="id_pracownika")
    
    return pracownicy

In [ ]:
pracownicy = sales()
pracownicy

,id_pracownika,imię_pracownika,nazwisko_pracownika,wiek_pracownika,adres_pracownika,telefon_pracownika,rola,pensja
2,1,Alutka,Myśliwiec,21,Kwiatowa 62,798973306,sprzedawca,3755
1,2,Nati,Lach,22,Magnolii 35,596989925,sprzedawca,3755
0,3,Misio,Bresiński,22,Głogowa 67,858448997,manager,6840
3,4,Fifol,Oszczepaliński,21,Boczna 85,576514962,logistyk,5257


In [ ]:
from sqlalchemy import create_engine, inspect
from sqlalchemy import URL

url_object = URL.create(
    "mysql+pymysql",
    username="team26",
    password="******",  # tu wpisać hasło i go nie commitować bo gitguardian was zje
    host="giniewicz.it",
    database="team26",
)

engine = create_engine(url_object)#'mysql+pymysql://team26:te@mzg@giniewicz.it:3306/team26')

connection = engine.connect()

connection.close()

ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
gry = gry.rename(columns={"ID":"id_gry", "NAZWA":"tytuł", "RODZAJ":"rodzaj", "CENA":"cena", "CZAS_GRY":"czas_gry", 
                         "MIN_GRACZY":"min_graczy", "MAX_GRACZY":"max_graczy", "MIN_WIEK":"min_wiek", "TURNIEJOWE":"turniejowe"})

In [ ]:
conn = engine.connect()
gry.to_sql("gry", engine, if_exists="replace", index = False)
pracownicy.to_sql("pracownicy", engine, if_exists="replace", index = False)
klienci.to_sql("klienci", engine, if_exists="replace", index=False)
outlet.to_sql("outlet", engine, if_exists="replace", index=False)
spichlerz_wynajem.to_sql("spichlerz_wynajem", engine, if_exists="replace", index=False)
wynajem.to_sql("wynajem", engine, if_exists="replace", index=False)
sklep.to_sql("sklep", engine, if_exists="replace", index=False)
spichlerz_sklep.to_sql("spichlerz_sklep", engine, if_exists="replace", index=False)
conn.close()